# Goals
+ summaries
+ number of and backfills per user
+ number of wont_process
+ how many language changers

`ssh -N dmca@cox.media.mit.edu -L 3308:cox.media.mit.edu:3306`


In [3]:
import sys
sys.path.append('/home/paprika/workspace/CivilServant-twitter/')
from utils.common import DbEngine
import os
import inspect
from app.models import TwitterStatus, TwitterFill, TwitterUser
import json
from sqlalchemy.sql.expression import func as sqlfunc
import datetime

BASE_DIR = os.path.join(os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))), "..")
ENV = 'production'
db_session = DbEngine(os.path.join(BASE_DIR, "config") + "/{env}.json".format(env=ENV)).new_session()

import pandas as pd
from sqlalchemy import create_engine
constr= 'mysql://civilservant@0.0.0.0:3308/civilservant_production'
con = create_engine(constr)
# con.execute('show tables;').fetchall()


In [4]:
should_be_filled = pd.read_sql("select * from twitter_users where user_rand <=0.5 and lang in ('en', 'en-gb', 'en-GB')", con)

In [37]:
should_be_filled['id'] = should_be_filled['id'].apply(int)

In [7]:
fill_records = pd.read_sql("select * from twitter_fills", con)

In [45]:
fill_records_users = pd.DataFrame(fill_records.groupby('user_id').size()).reset_index().rename_axis({0:"num_fills"},axis=1)

/home/paprika/workspace/py-flask/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  """Entry point for launching an IPython kernel.


In [50]:
user_fills = pd.merge(should_be_filled, fill_records_users, how='outer', left_on='id', right_on='user_id')

In [101]:
complete_user_cands = user_fills[(pd.notnull(user_fills['id'])) & (pd.notnull(user_fills['user_id']))]

In [103]:
complete_user_cands.shape

(16370, 12)

In [108]:
complete_user_cands_enough_fills = complete_user_cands[complete_user_cands['num_fills'] >= 22]

In [110]:
complete_user_cands_enough_fills.shape

(16341, 12)

In [111]:
complete_user_cands[complete_user_cands['num_fills'] <22].shape

(29, 12)

In [105]:
complete_user_cands_enough_fills.shape

(16146, 12)

In [113]:
complete_user_cands_enough_fills['num_fills'].value_counts()

23.0    8538
24.0    7608
22.0     195
Name: num_fills, dtype: int64

In [76]:
#who had fills which aren't in should fill
sample_mystery_filled = list(user_fills[pd.isnull(user_fills['id'])]['user_id'].apply(int).values)
sample_mystery_filled_str = [str(x) for x in sample_mystery_filled]

In [77]:
mystery_filled = pd.read_sql(f"select * from twitter_users where id in ({','.join(sample_mystery_filled_str)}) ", con)

/home/paprika/workspace/py-flask/lib/python3.6/site-packages/sqlalchemy/engine/default.py:508: Warning: (1292, "Truncated incorrect DOUBLE value: '<NOT_FOUND>_00900ll_1549959062589'")
  cursor.execute(statement, parameters)
/home/paprika/workspace/py-flask/lib/python3.6/site-packages/sqlalchemy/engine/default.py:508: Warning: (1292, "Truncated incorrect DOUBLE value: '<NOT_FOUND>_00moonmm_1550002438319'")
  cursor.execute(statement, parameters)
/home/paprika/workspace/py-flask/lib/python3.6/site-packages/sqlalchemy/engine/default.py:508: Warning: (1292, "Truncated incorrect DOUBLE value: '<NOT_FOUND>_01010109?protected_redirect=tr..._1549418634754'")
  cursor.execute(statement, parameters)
/home/paprika/workspace/py-flask/lib/python3.6/site-packages/sqlalchemy/engine/default.py:508: Warning: (1292, "Truncated incorrect DOUBLE value: '<NOT_FOUND>_01dtlz_1549115991045'")
  cursor.execute(statement, parameters)
/home/paprika/workspace/py-flask/lib/python3.6/site-packages/sqlalchemy/engine

In [112]:
mystery_filled.shape #these users all have different languages now

(81, 10)

In [81]:
mystery_unfilled  = user_fills[pd.isnull(user_fills['user_id'])]

In [83]:
mystery_unfilled.shape

(278, 12)

In [87]:
mystery_unfilled.head()

,id,not_found_id,screen_name,created_at,record_created_at,lang,user_state,CS_oldest_tweets_archived,last_attempted_process,user_rand,user_id,num_fills
22,1.000721e+18,None,king3prince352,2018-05-27 12:50:04,2019-02-14 14:14:13,en,1.0,1.0,2019-02-14 14:14:13,0.447045,NaN,NaN
109,1.004758e+18,None,saranvikaash2,2018-06-07 16:11:32,2019-01-11 18:56:04,en,1.0,2.0,2019-01-11 18:59:16,0.116588,NaN,NaN
116,1.004938e+18,None,tfc_off,2018-06-08 04:07:57,2019-01-11 18:55:49,en,1.0,2.0,2019-01-11 18:59:15,0.347281,NaN,NaN
247,1.009778e+18,<NOT_FOUND>_vmramkumar2_1551988407202,vmramkumar2,2018-06-21 12:38:39,2019-02-14 17:14:47,en,2.0,1.0,2019-02-14 17:14:47,0.037664,NaN,NaN
252,1.009929e+18,None,kp_six_293,2018-06-21 22:41:59,2019-02-14 11:13:19,en,1.0,1.0,2019-02-14 11:13:19,0.484708,NaN,NaN


In [92]:
mystery_unfilled['record_created_at'].apply(lambda ts: ts.date()).value_counts() #intersting that occured on two days

2019-02-14    224
2019-01-11     54
Name: record_created_at, dtype: int64

In [115]:
mystery_unfilled[mystery_unfilled['user_state']==1]

,id,not_found_id,screen_name,created_at,record_created_at,lang,user_state,CS_oldest_tweets_archived,last_attempted_process,user_rand,user_id,num_fills
22,1.000721e+18,None,king3prince352,2018-05-27 12:50:04,2019-02-14 14:14:13,en,1.0,1.0,2019-02-14 14:14:13,0.447045,NaN,NaN
109,1.004758e+18,None,saranvikaash2,2018-06-07 16:11:32,2019-01-11 18:56:04,en,1.0,2.0,2019-01-11 18:59:16,0.116588,NaN,NaN
116,1.004938e+18,None,tfc_off,2018-06-08 04:07:57,2019-01-11 18:55:49,en,1.0,2.0,2019-01-11 18:59:15,0.347281,NaN,NaN
252,1.009929e+18,None,kp_six_293,2018-06-21 22:41:59,2019-02-14 11:13:19,en,1.0,1.0,2019-02-14 11:13:19,0.484708,NaN,NaN
272,1.010537e+09,None,itslarisf,2012-12-14 06:34:39,2019-02-14 08:13:08,en,1.0,1.0,2019-02-14 08:13:08,0.096393,NaN,NaN
276,1.010776e+18,None,samthejamyo,2018-06-24 06:46:51,2019-02-14 17:14:47,en,1.0,1.0,2019-02-14 17:14:47,0.486690,NaN,NaN
318,1.012276e+18,None,movingirwins,2018-06-28 10:07:05,2019-02-14 08:13:06,en,1.0,1.0,2019-02-14 08:13:06,0.089458,NaN,NaN
355,1.013793e+18,None,shoh__kp,2018-07-02 14:34:42,2019-02-14 14:14:13,en,1.0,1.0,2019-02-14 14:14:13,0.379994,NaN,NaN
439,1.016940e+18,None,yyalkz,2018-07-11 06:57:34,2019-02-14 14:14:10,en,1.0,1.0,2019-02-14 14:14:10,0.117191,NaN,NaN
688,1.026455e+18,None,michieda_aya,2018-08-06 13:10:21,2019-02-14 14:14:11,en,1.0,1.0,2019-02-14 14:14:11,0.364396,NaN,NaN


In [95]:
mystery_unfilled['user_state'].value_counts()

1.0    263
3.0      7
4.0      5
2.0      3
Name: user_state, dtype: int64

In [98]:
mystery_unfilled[mystery_unfilled['user_state']!=1].sort_values('user_state') #4 is wont process, the other have not_found ids

,id,not_found_id,screen_name,created_at,record_created_at,lang,user_state,CS_oldest_tweets_archived,last_attempted_process,user_rand,user_id,num_fills
247,1.009778e+18,<NOT_FOUND>_vmramkumar2_1551988407202,vmramkumar2,2018-06-21 12:38:39,2019-02-14 17:14:47,en,2.0,1.0,2019-02-14 17:14:47,0.037664,NaN,NaN
2852,1.093075e+18,<NOT_FOUND>__cameo_rk_1550433114537,_cameo_rk,2019-02-06 09:14:38,2019-02-14 14:14:11,en,2.0,1.0,2019-02-14 14:14:11,0.213607,NaN,NaN
14958,8.955346e+17,<NOT_FOUND>_muskaan15__1551124306881,muskaan15_,2017-08-10 06:37:33,2019-02-14 17:14:48,en,2.0,1.0,2019-02-14 17:14:48,0.090239,NaN,NaN
1847,1.067934e+18,<NOT_FOUND>_tamilrockers_01_1547494461960,tamilrockers_01,2018-11-29 00:11:45,2019-01-11 18:55:58,en,3.0,2.0,2019-01-11 18:59:15,0.041074,NaN,NaN
3351,1.307436e+09,<NOT_FOUND>_traceylife_1551729254111,traceylife,2013-03-27 12:58:36,2019-02-14 08:13:07,en,3.0,1.0,2019-02-14 08:13:07,0.137036,NaN,NaN
3748,1.469993e+08,<NOT_FOUND>_rickthesizz1er_1550519488314,rickthesizz1er,2010-05-22 23:21:43,2019-02-14 08:13:14,en,3.0,1.0,2019-02-14 08:13:14,0.048442,NaN,NaN
6616,2.657943e+07,<NOT_FOUND>_djdonnied_1550519526685,djdonnied,2009-03-25 20:12:16,2019-02-14 08:13:08,en,3.0,1.0,2019-02-14 08:13:08,0.027175,NaN,NaN
8501,3.219255e+09,<NOT_FOUND>_loveumovies_1548790812012,loveumovies,2015-05-18 10:50:46,2019-01-11 18:55:51,en,3.0,2.0,2019-01-11 18:59:15,0.496116,NaN,NaN
9632,3.760950e+08,<NOT_FOUND>_prim_primmie_1551210895369,prim_primmie,2011-09-19 09:09:09,2019-02-14 14:14:11,en,3.0,1.0,2019-02-14 14:14:11,0.268401,NaN,NaN
12142,7.087539e+17,<NOT_FOUND>_arabiciggy_1550260335380,arabiciggy,2016-03-12 20:37:58,2019-02-14 17:14:55,en,3.0,1.0,2019-02-14 17:14:55,0.221282,NaN,NaN
